# IMPORT DE DONNEES

In [ ]:
#VISUALISATION ET IMPORT DE DONNEES
import os
import pickle
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt
import seaborn as sns



#SKLEARN FUNCTIONS
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV, KFold, cross_val_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import make_pipeline
import joblib


########################      PRE-PROCESSING      #############################


#ouverture et lecture des deux fichiers csv : frequences propres et entrées 
freq = pd.read_csv(open("testPOC.csv", "r"),
                    delimiter=",")
inputs = pd.read_csv(open("dictPOC.csv", "r"),
                    delimiter=",")
#2 dataFrames sont créés

#keep 1 column of freq


#Concatenation des deux dataframes dans le même DataFrame : data
datas = [inputs, freq]
datas = pd.concat(datas, axis=1)


def plot_correlation_matrix(data):
    corr = data.corr()
    sns.heatmap(corr, xticklabels=corr.columns, yticklabels=corr.columns)
    plt.show()
    print(corr)

# plot_correlation_matrix(pd.DataFrame(datas))

# d'apres la matrice de correlation, certaines entrées sont étroitement liées
# on va donc supprimer certaines de ces valeurs pour conserver :
# hauteur h, base b, la masse volumique rho, la longueur de la poutre L_tot
to_drop = ['NbElts', 'S', 'I', 'L', 'E','freq2','freq3','freq4','freq5','freq6','freq7','freq8']
datas = datas.drop(columns=to_drop)
#keep 10000 random values of datas


print(datas)
######################      FIN PRE-PROCESSING      ###########################




#######################      TRAIN_TEST_SPLIT      ############################
# 70% de la population sera allouée à l'apprentissage, 30 % pour le test
population_train = 0.7

# mélange et séparation de nos données en 2 datasets  
split_train, split_test = train_test_split(datas, train_size=population_train)

# On extrait les données qui serviront d'objectif à atteindre, soit ici les 
# 8 fréquences propres à prédire

entrees = ['L_tot','rho', 'h', 'b']
split_target_train = split_train.drop(columns=entrees)
split_target_test = split_test.drop(columns=entrees)


frequences = ["freq1"]
split_train = split_train.drop(columns=frequences)
split_test = split_test.drop(columns=frequences)

print("entrées train : \n",split_train)
print("target train : \n", split_target_train)

#split_train = entrees servant à entrainer le modèle
#split test = entrees servant à tester le modèle
#split_target_train = sorties d'entrainement du modèle
#split_target_test = sorties de test du modèle 


#######################      FIN TR_TST_SPLIT      ############################

# PROCESSING

In [ ]:
def PolynomialRegression(degree=2, **kwargs):
    return make_pipeline(PolynomialFeatures(degree), LinearRegression(**kwargs))

In [ ]:
"""my_grid_poly ={'polynomialfeatures__degree': [9],
                'linearregression__fit_intercept': [True, False],
                'linearregression__normalize': [True, False]}


poly_reg_model = PolynomialRegression(9, fit_intercept=True, normalize=True)
grid_search = GridSearchCV(poly_reg_model, my_grid_poly, cv=5, n_jobs=-1, verbose=2,)
#print best score and model
# Ravel plit_target_train
jesuisunfdp = grid_search.fit(split_train, split_target_train)
print(grid_search.best_score_)
print(grid_search.best_estimator_)
print(grid_search.best_params_)"""

In [ ]:
poly_reg_model = PolynomialRegression(9, fit_intercept=True, normalize=True)
poly_reg_model.fit(split_train, split_target_train)
print("",poly_reg_model.score(split_test, split_target_test))

In [ ]:
Y_pred = poly_reg_model.predict(split_test)
plt.figure(figsize=(12, 5))
plt.plot((Y_pred)[:80])
plt.plot((np.array(split_target_test)[:80]))

plt.legend(
    ['Y_grid_search', 'split_target_test'])
plt.show()


In [ ]:
from sklearn.pipeline import Pipeline


polynomial_features = PolynomialFeatures(degree=9)
linear_regression = LinearRegression(fit_intercept=True, normalize=True)
pipeline = Pipeline(
    [
        ("polynomial_features", polynomial_features),
        ("linear_regression", linear_regression),
    ]
)
pipeline.fit(split_train, split_target_train)
print("",pipeline.score(split_test, split_target_test))

In [ ]:
poly_reg_model = PolynomialRegression(2 , fit_intercept=True, normalize=True)
poly_reg_model.fit(split_train, split_target_train)
print("",poly_reg_model.score(split_test, split_target_test))

# PASSAGE A L'EVALUATION DU MODELE EN FONCTION DU NOMBRE DE DONNEES

In [ ]:
freq = pd.read_csv(open("test10el.csv", "r"),
                    delimiter=",")
inputs = pd.read_csv(open("dict10el.csv", "r"),
                    delimiter=",")
#2 dataFrames sont créés

#keep 1 column of freq


#Concatenation des deux dataframes dans le même DataFrame : data
datas = [inputs, freq]
datas = pd.concat(datas, axis=1)


def plot_correlation_matrix(data):
    corr = data.corr()
    sns.heatmap(corr, xticklabels=corr.columns, yticklabels=corr.columns)
    plt.show()
    print(corr)

# plot_correlation_matrix(pd.DataFrame(datas))

# d'apres la matrice de correlation, certaines entrées sont étroitement liées
# on va donc supprimer certaines de ces valeurs pour conserver :
# hauteur h, base b, la masse volumique rho, la longueur de la poutre L_tot
to_drop = ['NbElts', 'S', 'I', 'L', 'E','freq2','freq3','freq4','freq5','freq6','freq7','freq8']
datas = datas.drop(columns=to_drop)
#keep 10000 random values of datas


print(datas)
######################      FIN PRE-PROCESSING      ###########################


In [ ]:
# keep 10000 random values of datas
import time


score = []
nbr_values = []
exec_time = []

for i in range(1500,3500,100):

    new_datas = datas.sample(i)
    #######################      TRAIN_TEST_SPLIT      ############################
    # 70% de la population sera allouée à l'apprentissage, 30 % pour le test
    population_train = 0.7

    # mélange et séparation de nos données en 2 datasets  
    split_train, split_test = train_test_split(new_datas, train_size=population_train)

    # On extrait les données qui serviront d'objectif à atteindre, soit ici les 
    # 8 fréquences propres à prédire

    entrees = ['L_tot','rho', 'h', 'b']
    split_target_train = split_train.drop(columns=entrees)
    split_target_test = split_test.drop(columns=entrees)


    frequences = ["freq1"]
    split_train = split_train.drop(columns=frequences)
    split_test = split_test.drop(columns=frequences)
    start = time.time()
    poly_reg_model = PolynomialRegression(9 , fit_intercept=True, normalize=True)
    poly_reg_model.fit(split_train, split_target_train)
    end = (time.time() - start)
    score.append(poly_reg_model.score(split_test, split_target_test))
    nbr_values.append(i)
    exec_time.append(time.time() - start)

plt.figure(figsize=(12, 5))
plt.plot(nbr_values, score)
plt.plot(nbr_values, exec_time)
plt.legend(
    ['score', 'exec_time'])
plt.show()

print(score)
print(exec_time)
print(nbr_values)

Y_pred = poly_reg_model.predict(split_test)
plt.figure(figsize=(12, 5))
plt.plot((Y_pred)[:80])
plt.plot((np.array(split_target_test)[:80]))

plt.legend(
    ['Y_grid_search', 'split_target_test'])
plt.show()

# show best ratio score / exec_time





    





## Entrainement 100 elements finnis


In [ ]:
freq = pd.read_csv(open("test100.csv", "r"),
                    delimiter=",")
inputs = pd.read_csv(open("dict100.csv", "r"),
                    delimiter=",")

# delete last column of inputs
#2 dataFrames sont créés

#keep 1 column of freq


#Concatenation des deux dataframes dans le même DataFrame : data
datas = [inputs, freq]
datas = pd.concat(datas, axis=1)


def plot_correlation_matrix(data):
    corr = data.corr()
    sns.heatmap(corr, xticklabels=corr.columns, yticklabels=corr.columns)
    plt.show()
    print(corr)

# plot_correlation_matrix(pd.DataFrame(datas))

# d'apres la matrice de correlation, certaines entrées sont étroitement liées
# on va donc supprimer certaines de ces valeurs pour conserver :
# hauteur h, base b, la masse volumique rho, la longueur de la poutre L_tot
to_drop = ['NbElts', 'S', 'I', 'L', 'E','freq2','freq3','freq4','freq5','freq6','freq7','freq8','Mat']
datas = datas.drop(columns=to_drop)
#keep 10000 random values of datas


print(datas)
######################      FIN PRE-PROCESSING      ###########################

In [ ]:
import time


score = []
nbr_values = []
exec_time = []

for i in range(2500,5000,100):

    new_datas = datas.sample(i)
    #######################      TRAIN_TEST_SPLIT      ############################
    # 70% de la population sera allouée à l'apprentissage, 30 % pour le test
    population_train = 0.7

    # mélange et séparation de nos données en 2 datasets  
    split_train, split_test = train_test_split(new_datas, train_size=population_train)

    # On extrait les données qui serviront d'objectif à atteindre, soit ici les 
    # 8 fréquences propres à prédire

    entrees = ['L_tot','rho', 'h', 'b']
    split_target_train = split_train.drop(columns=entrees)
    split_target_test = split_test.drop(columns=entrees)


    frequences = ["freq1"]
    split_train = split_train.drop(columns=frequences)
    split_test = split_test.drop(columns=frequences)
    start = time.time()
    poly_reg_model = PolynomialRegression(9 , fit_intercept=True, normalize=True)
    poly_reg_model.fit(split_train, split_target_train)
    end = (time.time() - start)
    score.append(poly_reg_model.score(split_test, split_target_test))
    nbr_values.append(i)
    exec_time.append(time.time() - start)

plt.figure(figsize=(12, 5))
plt.plot(nbr_values, score)
plt.plot(nbr_values, exec_time)
plt.legend(
    ['score', 'exec_time'])
plt.show()

print(score)
print(exec_time)
print(nbr_values)

Y_pred = poly_reg_model.predict(split_test)
plt.figure(figsize=(12, 5))
plt.plot((Y_pred)[:80])
plt.plot((np.array(split_target_test)[:80]))

plt.legend(
    ['Y_grid_search', 'split_target_test'])
plt.show()

for i in range(0, len(score)):
    score[i] = score[i] / exec_time[i]
  

plt.figure(figsize=(12, 5))
plt.plot(nbr_values, score)
plt.legend(
    ['score / exec_time'])
plt.show()






# Entrainement des modèles pour les autres formes de poutres


## Load et gestion des données

In [ ]:
freq_cercle = pd.read_csv(open("gen_data/test_Cercle.csv", "r"),delimiter=",")
input_cercle = pd.read_csv(open("gen_data/dict_Cercle.csv", "r"),delimiter=",")

freq_cercle_creux = pd.read_csv(open("gen_data/test_Cercle_Creux.csv", "r"),delimiter=",")
input_cercle_creux = pd.read_csv(open("gen_data/dict_Cercle_Creux.csv", "r"),delimiter=",")

freq_rectangle_creux = pd.read_csv(open("gen_data/test_Rectangle_Creux.csv", "r"),delimiter=",")
input_rectangle_creux = pd.read_csv(open("gen_data/dict_Rectangle_Creux.csv", "r"),delimiter=",")

freq_rectangle = pd.read_csv(open("test10el.csv", "r"),delimiter=",")
input_rectangle = pd.read_csv(open("dict10el.csv", "r"),delimiter=",")

freq_rectangle100el = pd.read_csv(open("test100.csv", "r"),delimiter=",")
input_rectangle100el = pd.read_csv(open("dict100.csv", "r"),delimiter=",")

data_cercle = [input_cercle, freq_cercle]
data_cercle = pd.concat(data_cercle, axis=1)

data_cercle_creux = [input_cercle_creux, freq_cercle_creux]
data_cercle_creux = pd.concat(data_cercle_creux, axis=1)

data_rectangle_creux = [input_rectangle_creux, freq_rectangle_creux]
data_rectangle_creux = pd.concat(data_rectangle_creux, axis=1)

data_rectangle = [input_rectangle, freq_rectangle]
data_rectangle = pd.concat(data_rectangle, axis=1)



to_drop = ['NbElts', 'S', 'I', 'L', 'E','freq2','freq3','freq4','freq5','freq6','freq7','freq8','Mat']
to_drop_sans_mat = ['NbElts', 'S', 'I', 'L', 'E','freq2','freq3','freq4','freq5','freq6','freq7','freq8']
freq1 = frequences

data_cercle = data_cercle.drop(columns=to_drop)
data_cercle_creux = data_cercle_creux.drop(columns=to_drop)
data_rectangle_creux = data_rectangle_creux.drop(columns=to_drop)
data_rectangle = data_rectangle.drop(columns=to_drop_sans_mat)


split_train_cercle,split_test_cercle = train_test_split(data_cercle, train_size=population_train)
entrees_cercle = ['L_tot','rho', 'r']
split_target_train_cercle = split_train_cercle.drop(columns=entrees_cercle)
split_target_test_cercle = split_test_cercle.drop(columns=entrees_cercle)
split_train_cercle.head(10)
print(split_train_cercle)



split_train_cercle_creux,split_test_cercle_creux = train_test_split(data_cercle_creux, train_size=population_train)
entrees_cercle_creux = ['L_tot','rho', 'r_ext','r_int']
split_target_train_cercle_creux = split_train_cercle_creux.drop(columns=entrees_cercle_creux)
split_target_test_cercle_creux = split_test_cercle_creux.drop(columns=entrees_cercle_creux)
split_train_cercle_creux = split_train_cercle_creux.drop(columns=freq1)
split_test_cercle_creux = split_test_cercle_creux.drop(columns=freq1)


entrees_rectangle_creux = ['L_tot','rho', 'h_ext','b_ext','h_int','b_int']
split_train_rectangle_creux,split_test_rectangle_creux = train_test_split(data_rectangle_creux, train_size=population_train)
split_target_train_rectangle_creux = split_train_rectangle_creux.drop(columns=entrees_rectangle_creux)
split_target_test_rectangle_creux = split_test_rectangle_creux.drop(columns=entrees_rectangle_creux)
split_train_rectangle_creux = split_train_rectangle_creux.drop(columns=freq1)
split_test_rectangle_creux = split_test_rectangle_creux.drop(columns=freq1)


entrees_rectangle = ['L_tot','rho', 'h','b']
split_train_rectangle,split_test_rectangle = train_test_split(data_rectangle, train_size=population_train)
split_target_train_rectangle = split_train_rectangle.drop(columns=entrees_rectangle)
split_target_test_rectangle = split_test_rectangle.drop(columns=entrees_rectangle)
split_train_rectangle = split_train_rectangle.drop(columns=freq1)
split_test_rectangle = split_test_rectangle.drop(columns=freq1)


## Entrainement des modèles

In [ ]:
poly_reg_cercle = PolynomialRegression(9, fit_intercept=True, normalize=True)
poly_reg_cercle.fit(split_train_cercle, split_target_train_cercle)
poly_reg_cercle_creux = PolynomialRegression(9, fit_intercept=True, normalize=True)
poly_reg_cercle_creux.fit(split_train_cercle_creux, split_target_train_cercle_creux)
poly_reg_rectangle_creux = PolynomialRegression(9, fit_intercept=True, normalize=True)
poly_reg_rectangle_creux.fit(split_train_rectangle_creux, split_target_train_rectangle_creux)
poly_reg_rectangle = PolynomialRegression(9, fit_intercept=True, normalize=True)
poly_reg_rectangle.fit(split_train_rectangle, split_target_train_rectangle)

Y_pred_cercle = poly_reg_cercle.predict(split_test_cercle)
Y_pred_cercle_creux = poly_reg_cercle_creux.predict(split_test_cercle_creux)
Y_pred_rectangle_creux = poly_reg_rectangle_creux.predict(split_test_rectangle_creux)
Y_pred_rectangle = poly_reg_rectangle.predict(split_test_rectangle)

# print score for each modèle
print("Cercle : ",poly_reg_cercle.score(split_test_cercle, split_target_test_cercle))
print("Cercle creux",poly_reg_cercle_creux.score(split_test_cercle_creux, split_target_test_cercle_creux))
print("Rectangle creuc",poly_reg_rectangle_creux.score(split_test_rectangle_creux, split_target_test_rectangle_creux))
print("Rectangle",poly_reg_rectangle.score(split_test_rectangle, split_target_test_rectangle))



In [ ]:
split_train_rectangle.head(10)

In [ ]:
# grid search for cercle_creux

def PolynomialRegression(degree=2, **kwargs):
    return make_pipeline(PolynomialFeatures(degree), LinearRegression(**kwargs))

"""my_grid_poly ={'polynomialfeatures__degree': [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20],
                'linearregression__fit_intercept': [True],
                'linearregression__normalize': [True]}
grid = GridSearchCV(PolynomialRegression(), my_grid_poly, cv=2,verbose=1)
grid.fit(split_train_cercle_creux, split_target_train_cercle_creux)
print(grid.best_params_)
print(grid.best_score_)"""
# best param degré = 6 true true for 0.97 best score 

my_grid_rf ={'n_estimators': [80],
                'max_depth': [19,],
                'min_samples_split': [2,3,4,5,6,7,8,9,10],
              }
                
grid = GridSearchCV(RandomForestRegressor(), my_grid_rf, cv=2,verbose=1)
grid.fit(split_train_cercle_creux, split_target_train_cercle_creux)
print(grid.best_params_)
print(grid.best_score_)

In [11]:
"""# plot prediction
plt.figure(figsize=(12, 5))
plt.plot((Y_pred_cercle)[:80])
plt.plot((np.array(split_target_test_cercle)[:80]))

plt.legend(
    ['Y_grid_search', 'split_target_test'])
plt.show()

plt.figure(figsize=(12, 5))
plt.plot((Y_pred_cercle_creux)[:80])
plt.plot((np.array(split_target_test_cercle_creux)[:80]))

plt.legend(
    ['Y_grid_search', 'split_target_test'])
plt.show()

plt.figure(figsize=(12, 5))
plt.plot((Y_pred_rectangle_creux)[:80])
plt.plot((np.array(split_target_test_rectangle_creux)[:80]))

plt.legend(
    ['Y_grid_search', 'split_target_test'])
plt.show()"""

from other_freqs import gen_model_Rectangle_creux, gen_model_Cercle_creux, gen_model_cercle, gen_model_rect

reg_lin_rect = gen_model_rect()
reg_lin_cercle = gen_model_cercle()
reg_lin_rect_creux = gen_model_Rectangle_creux()
reg_lin_cercle_creux = gen_model_Cercle_creux()

ImportError: cannot import name 'gen_model_Rectangle_creux' from 'other_freqs' (C:\Dev\5A\apprentissage_automatique\projetIA\other_freqs.py)

In [ ]:
# save all 4 models with pickle
with open('model_cercle.pkl', 'wb') as f:
    pickle.dump(poly_reg_cercle, f)
with open('model_cercle_creux.pkl', 'wb') as f:
    pickle.dump(poly_reg_cercle_creux, f)
with open('model_rectangle_creux.pkl', 'wb') as f:
    pickle.dump(poly_reg_rectangle_creux, f)
with open('model_rectangle.pkl', 'wb') as f:
    pickle.dump(poly_reg_rectangle, f)

# Entrainement des modèles pour les autres formes de poutres


## Load et gestion des données

In [55]:
freq_cercle = pd.read_csv(open("gen_data/test_Cercle.csv", "r"),delimiter=",")
input_cercle = pd.read_csv(open("gen_data/dict_Cercle.csv", "r"),delimiter=",")

freq_cercle_creux = pd.read_csv(open("gen_data/test_Cercle_Creux.csv", "r"),delimiter=",")
input_cercle_creux = pd.read_csv(open("gen_data/dict_Cercle_Creux.csv", "r"),delimiter=",")

freq_rectangle_creux = pd.read_csv(open("gen_data/test_Rectangle_Creux.csv", "r"),delimiter=",")
input_rectangle_creux = pd.read_csv(open("gen_data/dict_Rectangle_Creux.csv", "r"),delimiter=",")

freq_rectangle = pd.read_csv(open("test10el.csv", "r"),delimiter=",")
input_rectangle = pd.read_csv(open("dict10el.csv", "r"),delimiter=",")

freq_rectangle100el = pd.read_csv(open("test100.csv", "r"),delimiter=",")
input_rectangle100el = pd.read_csv(open("dict100.csv", "r"),delimiter=",")

data_cercle = [input_cercle, freq_cercle]
data_cercle = pd.concat(data_cercle, axis=1)

data_cercle_creux = [input_cercle_creux, freq_cercle_creux]
data_cercle_creux = pd.concat(data_cercle_creux, axis=1)

data_rectangle_creux = [input_rectangle_creux, freq_rectangle_creux]
data_rectangle_creux = pd.concat(data_rectangle_creux, axis=1)

data_rectangle = [input_rectangle, freq_rectangle]
data_rectangle = pd.concat(data_rectangle, axis=1)



to_drop = ['NbElts', 'S', 'I', 'L', 'E','freq2','freq3','freq4','freq5','freq6','freq7','freq8','Mat']
to_drop_sans_mat = ['NbElts', 'S', 'I', 'L', 'E','freq2','freq3','freq4','freq5','freq6','freq7','freq8']
freq1 = frequences

data_cercle = data_cercle.drop(columns=to_drop)
data_cercle_creux = data_cercle_creux.drop(columns=to_drop)
data_rectangle_creux = data_rectangle_creux.drop(columns=to_drop)
data_rectangle = data_rectangle.drop(columns=to_drop_sans_mat)


split_train_cercle,split_test_cercle = train_test_split(data_cercle, train_size=population_train)
entrees_cercle = ['L_tot','rho', 'r']
split_target_train_cercle = split_train_cercle.drop(columns=entrees_cercle)
split_target_test_cercle = split_test_cercle.drop(columns=entrees_cercle)
split_train_cercle.head(10)
print(split_train_cercle)



split_train_cercle_creux,split_test_cercle_creux = train_test_split(data_cercle_creux, train_size=population_train)
entrees_cercle_creux = ['L_tot','rho', 'r_ext','r_int']
split_target_train_cercle_creux = split_train_cercle_creux.drop(columns=entrees_cercle_creux)
split_target_test_cercle_creux = split_test_cercle_creux.drop(columns=entrees_cercle_creux)
split_train_cercle_creux = split_train_cercle_creux.drop(columns=freq1)
split_test_cercle_creux = split_test_cercle_creux.drop(columns=freq1)


entrees_rectangle_creux = ['L_tot','rho', 'h_ext','b_ext','h_int','b_int']
split_train_rectangle_creux,split_test_rectangle_creux = train_test_split(data_rectangle_creux, train_size=population_train)
split_target_train_rectangle_creux = split_train_rectangle_creux.drop(columns=entrees_rectangle_creux)
split_target_test_rectangle_creux = split_test_rectangle_creux.drop(columns=entrees_rectangle_creux)
split_train_rectangle_creux = split_train_rectangle_creux.drop(columns=freq1)
split_test_rectangle_creux = split_test_rectangle_creux.drop(columns=freq1)


entrees_rectangle = ['L_tot','rho', 'h','b']
split_train_rectangle,split_test_rectangle = train_test_split(data_rectangle, train_size=population_train)
split_target_train_rectangle = split_train_rectangle.drop(columns=entrees_rectangle)
split_target_test_rectangle = split_test_rectangle.drop(columns=entrees_rectangle)
split_train_rectangle = split_train_rectangle.drop(columns=freq1)
split_test_rectangle = split_test_rectangle.drop(columns=freq1)


         L_tot   rho         r        freq1
449   0.514264  8900  0.044380   175.255997
1373  0.577621  4510  0.027447   115.722000
962   0.153427  2700  0.004409   264.896088
960   0.912649  8900  0.016348    20.498699
2132  0.646620  8900  0.031607    78.946899
...        ...   ...       ...          ...
258   0.906701  2700  0.076221   131.139099
1561  0.512010  2500  0.037824   212.083099
233   0.128880  7850  0.012527  1091.419800
272   0.491170  8900  0.041239   178.527893
415   0.330031  8900  0.010597   101.608597

[1750 rows x 4 columns]


## Entrainement des modèles

In [56]:
poly_reg_cercle = PolynomialRegression(9, fit_intercept=True, normalize=True)
poly_reg_cercle.fit(split_train_cercle, split_target_train_cercle)
poly_reg_cercle_creux = PolynomialRegression(9, fit_intercept=True, normalize=True)
poly_reg_cercle_creux.fit(split_train_cercle_creux, split_target_train_cercle_creux)
poly_reg_rectangle_creux = PolynomialRegression(9, fit_intercept=True, normalize=True)
poly_reg_rectangle_creux.fit(split_train_rectangle_creux, split_target_train_rectangle_creux)
poly_reg_rectangle = PolynomialRegression(9, fit_intercept=True, normalize=True)
poly_reg_rectangle.fit(split_train_rectangle, split_target_train_rectangle)

Y_pred_cercle = poly_reg_cercle.predict(split_test_cercle)
Y_pred_cercle_creux = poly_reg_cercle_creux.predict(split_test_cercle_creux)
Y_pred_rectangle_creux = poly_reg_rectangle_creux.predict(split_test_rectangle_creux)
Y_pred_rectangle = poly_reg_rectangle.predict(split_test_rectangle)

# print score for each modèle
print("Cercle : ",poly_reg_cercle.score(split_test_cercle, split_target_test_cercle))
print("Cercle creux",poly_reg_cercle_creux.score(split_test_cercle_creux, split_target_test_cercle_creux))
print("Rectangle creuc",poly_reg_rectangle_creux.score(split_test_rectangle_creux, split_target_test_rectangle_creux))
print("Rectangle",poly_reg_rectangle.score(split_test_rectangle, split_target_test_rectangle))



c:\Users\enzoh\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), LinearRegression())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)


  warnings.warn(
c:\Users\enzoh\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous be

Cercle :  1.0
Cercle creux 0.6955744977851921
Rectangle creuc -6055818.442892607
Rectangle 0.9991612436631695


In [57]:
split_train_rectangle.head(10)

,L_tot,rho,h,b
13412,0.477909,2400,0.047055,0.029576
27535,0.460209,2700,0.006365,0.021289
89717,0.920023,7800,0.065778,0.048732
48235,0.451220,7850,0.017114,0.025872
21381,0.984723,7850,0.015821,0.055604
92054,0.441646,2700,0.012825,0.031244
57135,0.763468,8900,0.042613,0.068800
87646,0.629477,8900,0.047380,0.036035
40190,0.226298,7850,0.004465,0.009514
7029,0.245365,2700,0.004439,0.020329


In [71]:
# grid search for cercle_creux

def PolynomialRegression(degree=2, **kwargs):
    return make_pipeline(PolynomialFeatures(degree), LinearRegression(**kwargs))

"""my_grid_poly ={'polynomialfeatures__degree': [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20],
                'linearregression__fit_intercept': [True],
                'linearregression__normalize': [True]}
grid = GridSearchCV(PolynomialRegression(), my_grid_poly, cv=2,verbose=1)
grid.fit(split_train_cercle_creux, split_target_train_cercle_creux)
print(grid.best_params_)
print(grid.best_score_)"""
# best param degré = 6 true true for 0.97 best score 

my_grid_rf ={'n_estimators': [80],
                'max_depth': [19,],
                'min_samples_split': [2,3,4,5,6,7,8,9,10],
              }
                
grid = GridSearchCV(RandomForestRegressor(), my_grid_rf, cv=2,verbose=1)
grid.fit(split_train_cercle_creux, split_target_train_cercle_creux)
print(grid.best_params_)
print(grid.best_score_)

Fitting 2 folds for each of 20 candidates, totalling 40 fits


c:\Users\enzoh\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
c:\Users\enzoh\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
c:\Users\enzoh\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
c:\Users\enzoh\AppData\Local\Programs\Py

{'max_depth': 19, 'n_estimators': 80}
0.9378586418994692


In [ ]:
"""# plot prediction
plt.figure(figsize=(12, 5))
plt.plot((Y_pred_cercle)[:80])
plt.plot((np.array(split_target_test_cercle)[:80]))

plt.legend(
    ['Y_grid_search', 'split_target_test'])
plt.show()

plt.figure(figsize=(12, 5))
plt.plot((Y_pred_cercle_creux)[:80])
plt.plot((np.array(split_target_test_cercle_creux)[:80]))

plt.legend(
    ['Y_grid_search', 'split_target_test'])
plt.show()

plt.figure(figsize=(12, 5))
plt.plot((Y_pred_rectangle_creux)[:80])
plt.plot((np.array(split_target_test_rectangle_creux)[:80]))

plt.legend(
    ['Y_grid_search', 'split_target_test'])
plt.show()"""

"# plot prediction\nplt.figure(figsize=(12, 5))\nplt.plot((Y_pred_cercle)[:80])\nplt.plot((np.array(split_target_test_cercle)[:80]))\n\nplt.legend(\n    ['Y_grid_search', 'split_target_test'])\nplt.show()\n\nplt.figure(figsize=(12, 5))\nplt.plot((Y_pred_cercle_creux)[:80])\nplt.plot((np.array(split_target_test_cercle_creux)[:80]))\n\nplt.legend(\n    ['Y_grid_search', 'split_target_test'])\nplt.show()\n\nplt.figure(figsize=(12, 5))\nplt.plot((Y_pred_rectangle_creux)[:80])\nplt.plot((np.array(split_target_test_rectangle_creux)[:80]))\n\nplt.legend(\n    ['Y_grid_search', 'split_target_test'])\nplt.show()"

In [ ]:
# save all 4 models with pickle
with open('model_cercle.pkl', 'wb') as f:
    pickle.dump(poly_reg_cercle, f)
with open('model_cercle_creux.pkl', 'wb') as f:
    pickle.dump(poly_reg_cercle_creux, f)
with open('model_rectangle_creux.pkl', 'wb') as f:
    pickle.dump(poly_reg_rectangle_creux, f)
with open('model_rectangle.pkl', 'wb') as f:
    pickle.dump(poly_reg_rectangle, f)

